In [1]:
from timeit import timeit

from src.params import *

from src.nws import NWSClient

first_time = timeit(lambda: NWSClient().get_location_info(CENTRAL_PARK[0], CENTRAL_PARK[1]), number=2)
print(f"First time: {first_time}")

client = NWSClient()
client.get_location_info(CENTRAL_PARK[0], CENTRAL_PARK[1])

second_time = timeit(lambda: client.get_location_info(CENTRAL_PARK[0], CENTRAL_PARK[1]), number=2)
print(f"Second time: {second_time}")

First time: 0.3140354090010078
Second time: 6.309299897111487e-05


In [1]:
from pprint import pprint
import requests

from src.params import *
from src.nws import NWSClient
# intForecastFeatureFlags

client = NWSClient()
url = client.get_forecast_url(*CENTRAL_PARK)
res1 = requests.get(url).json()
url = client.get_hourly_forecast_url(*CENTRAL_PARK)
res2 = requests.get(url).json()

# max(res['properties']['periods']['temperature']['values'], key=lambda x: x['validTime'])
# client.get_location_info(CENTRAL_PARK[0], CENTRAL_PARK[1])
forecast = client.get_forecast_data(*CENTRAL_PARK)
hourly_forecast = client.get_hourly_forecast_data(*CENTRAL_PARK)
# pprint(client.get_hourly_forecast_data(*CENTRAL_PARK))

# print(forecast)
# pprint(res)

In [5]:
res1

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-73.9830679, 40.7887016],
    [-73.9875421, 40.767019000000005],
    [-73.9589182, 40.763629400000006],
    [-73.954438, 40.78531160000001],
    [-73.9830679, 40.7887016]]]},
 'properties': {'units': 'us',
  'forecastGenerator': 'BaselineForecastGenerator',
  'generatedAt': '2024-11-10T18:46:03+00:00',
  'updateTime': '2024-11-10T17:28:21+00:00',
  'validTimes': '2024-11-10T11:00:00+00:00/P8DT2H',
  'elevation': {'unitCode': 'wmoUnit:m', 'value': 24.9936},
  'periods': [{'number': 1,
    'name': 'This Afternoon',
    'startTime': '2024-11-10T13:00:00-05:00',
    'endTime': '2024-11-10T18:00:00-05:00',
    'isDaytime': True,
    '

In [2]:
pprint(forecast)

[Forecast(lat=40.7833546,
          lon=-73.9649732,
          generated_at=datetime.datetime(2024, 11, 10, 18, 46, 3, tzinfo=datetime.timezone.utc),
          update_time=datetime.datetime(2024, 11, 10, 17, 28, 21, tzinfo=datetime.timezone.utc),
          start_time=datetime.datetime(2024, 11, 10, 13, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=68400))),
          end_time=datetime.datetime(2024, 11, 10, 18, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=68400))),
          date=datetime.date(2024, 11, 10),
          temperature=60,
          temperature_type=<TemperatureType.HIGH: 'high'>),
 Forecast(lat=40.7833546,
          lon=-73.9649732,
          generated_at=datetime.datetime(2024, 11, 10, 18, 46, 3, tzinfo=datetime.timezone.utc),
          update_time=datetime.datetime(2024, 11, 10, 17, 28, 21, tzinfo=datetime.timezone.utc),
          start_time=datetime.datetime(2024, 11, 10, 18, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, sec

In [3]:
pprint(client.get_hourly_forecast_data(*CENTRAL_PARK))

[Forecast(lat=40.7833546,
          lon=-73.9649732,
          generated_at=datetime.datetime(2024, 11, 10, 19, 21, 31, tzinfo=datetime.timezone.utc),
          update_time=datetime.datetime(2024, 11, 10, 17, 28, 21, tzinfo=datetime.timezone.utc),
          start_time=datetime.datetime(2024, 11, 10, 14, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=68400))),
          end_time=datetime.datetime(2024, 11, 10, 15, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=68400))),
          date=datetime.date(2024, 11, 10),
          temperature=59,
          temperature_type=<TemperatureType.HIGH: 'high'>),
 Forecast(lat=40.7833546,
          lon=-73.9649732,
          generated_at=datetime.datetime(2024, 11, 10, 19, 21, 31, tzinfo=datetime.timezone.utc),
          update_time=datetime.datetime(2024, 11, 10, 17, 28, 21, tzinfo=datetime.timezone.utc),
          start_time=datetime.datetime(2024, 11, 10, 15, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, s

In [6]:
from urlpath import URL
URL("https://api.weather.gov") / "points" / "39.7456,-97.0892" / "forecast"

URL('https://api.weather.gov/points/39.7456,-97.0892/forecast')

In [1]:
lst = ["ca",
"co",
"ct",
"de",
"hi",
"il",
"md",
"ma",
"nj",
"nm",
"ny",
"or",
"ri",
"vt",
"va",
"wa",]

lst = [x.upper() for x in lst]
print(lst)

['CA', 'CO', 'CT', 'DE', 'HI', 'IL', 'MD', 'MA', 'NJ', 'NM', 'NY', 'OR', 'RI', 'VT', 'VA', 'WA']
